# Лабораторная работа № 2
### Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.


## - Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

## - Инициализация датасета
#### (Датасет представляет собой данные о прошедших матчах в Бразильском CS:GO)

In [3]:
file_path = r'C:\Users\thebi\tb_lobby_stats_player.csv'

data = pd.read_csv(file_path)

num_rows, num_cols = data.shape
print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_cols}")


Количество строк: 184152
Количество столбцов: 38


In [4]:
print(data.columns)

Index(['idLobbyGame', 'idPlayer', 'idRoom', 'qtKill', 'qtAssist', 'qtDeath',
       'qtHs', 'qtBombeDefuse', 'qtBombePlant', 'qtTk', 'qtTkAssist',
       'qt1Kill', 'qt2Kill', 'qt3Kill', 'qt4Kill', 'qt5Kill', 'qtPlusKill',
       'qtFirstKill', 'vlDamage', 'qtHits', 'qtShots', 'qtLastAlive',
       'qtClutchWon', 'qtRoundsPlayed', 'descMapName', 'vlLevel', 'qtSurvived',
       'qtTrade', 'qtFlashAssist', 'qtHitHeadshot', 'qtHitChest',
       'qtHitStomach', 'qtHitLeftAtm', 'qtHitRightArm', 'qtHitLeftLeg',
       'qtHitRightLeg', 'flWinner', 'dtCreatedAt'],
      dtype='object')


## - Удаление столбцов с необрабатываемыми типами данных или просто не влияющих на исход матча

In [5]:
columns_to_drop = ['idRoom', 'idPlayer', 'idLobbyGame', 'qtFlashAssist', 'qtHitHeadshot', 'qtHitChest',
       'qtHitStomach', 'qtHitLeftAtm', 'qtHitRightArm', 'qtHitLeftLeg',
       'qtHitRightLeg', 'dtCreatedAt', 'descMapName']
data = data.drop(columns=columns_to_drop)

In [6]:
print(data.columns)

Index(['qtKill', 'qtAssist', 'qtDeath', 'qtHs', 'qtBombeDefuse',
       'qtBombePlant', 'qtTk', 'qtTkAssist', 'qt1Kill', 'qt2Kill', 'qt3Kill',
       'qt4Kill', 'qt5Kill', 'qtPlusKill', 'qtFirstKill', 'vlDamage', 'qtHits',
       'qtShots', 'qtLastAlive', 'qtClutchWon', 'qtRoundsPlayed', 'vlLevel',
       'qtSurvived', 'qtTrade', 'flWinner'],
      dtype='object')


## - Поиск пустых значений в датасете и удаление соответствующих строк

In [9]:
# Урезка датасета
data = data.sample(n=50000, random_state=42)

empty_columns = data.columns[data.isnull().any()]
empty_columns_count = data[empty_columns].isnull().sum()

# Вывод столбцов с пустыми значениями и их количество
if len(empty_columns) > 0:
    print("Столбцы с пустыми значениями и их количество:")
    for column in empty_columns:
        print(f"Столбец '{column}': {empty_columns_count[column]} пустых значений")
    print()

    # Удаление строк с пустыми значениями
    data_cleaned = data.dropna()
    print(f"Удалено строк с пустыми значениями: {len(data) - len(data_cleaned)}")
else:
    print("В датасете нет пустых значений")

Столбцы с пустыми значениями и их количество:
Столбец 'qtTk': 37 пустых значений
Столбец 'qtTkAssist': 37 пустых значений
Столбец 'qtHits': 37 пустых значений
Столбец 'qtLastAlive': 37 пустых значений
Столбец 'qtSurvived': 189 пустых значений
Столбец 'qtTrade': 189 пустых значений

Удалено строк с пустыми значениями: 189


## - Поиск столбцов с нечисловыми значениями

In [10]:
non_numeric_columns = data_cleaned.select_dtypes(exclude=['number']).columns.tolist()

if len(non_numeric_columns) > 0:
    print("Столбцы с нечисловыми значениями:")
    for column in non_numeric_columns:
        print(column)
else:
    print("Все столбцы содержат числовые значения")

# Подсчет количества строк и столбцов
num_rows, num_cols = data_cleaned.shape
print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_cols}")

Все столбцы содержат числовые значения
Количество строк: 49811
Количество столбцов: 25


## - Поиск лучших гиперпараметров, а также разделение датасета

In [8]:
X = data_cleaned.drop(columns=['flWinner'])
y = data_cleaned['flWinner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

param_grid = {'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
              'C': [0.001, 0.01, 0.1, 1, 10],
              'degree': [1, 2, 3, 4, 5, 6, 7]}  

svm_grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

svm_grid.fit(X_test, y_test)

best_params = svm_grid.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']
best_degree = best_params['degree']

print(f"Лучшие параметры: Ядро = {best_kernel}, C = {best_C}, Degree = {best_degree}")

KeyboardInterrupt: 

## - Обучение модели 

In [12]:
svm_model = SVC(kernel='linear', degree=1, C=0.1) 

# Обучение модели
svm_model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = svm_model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)

print(f'Точность модели: {accuracy}')
print(report)

# Кросс-валидация
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm_model, X_train, y_train, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')

Точность модели: 0.797741530740276
              precision    recall  f1-score   support

           0       0.79      0.81      0.80     10083
           1       0.80      0.78      0.79      9842

    accuracy                           0.80     19925
   macro avg       0.80      0.80      0.80     19925
weighted avg       0.80      0.80      0.80     19925

Средняя точность перекрестной проверки: 0.8013117138615391


Исследована модель машины опорных векторов (SVM) с линейным ядром. 
Модель обучена на тренировочном наборе данных и использована для предсказания на тестовом наборе данных. 
Для оценки качества модели использованы метрики точности, матрица ошибок, полнота и f1-мера. 
Модель показала точность прогноза около 80%, а также неплохие значения полноты и f1-меры. 
Средняя точность перекрестной проверки составила около 80%, что подтверждает хорошую обобщающую способность модели. 
Таким образом, модель SVM с линейным ядром показала хорошие результаты в задаче классификации и может быть рекомендована для дальнейшего использования.